In [ ]:
# @title Cell 1: CASME II PoolFormer Infrastructure Configuration

# File: 02_05_PoolFormer_CASME2_AF_Cell1.py
# Location: experiments/02_05_PoolFormer_CASME2-AF.ipynb
# Purpose: PoolFormer baseline for CASME II micro-expression recognition with advanced class weight optimization and focal loss

# Mount Google Drive
from google.colab import drive
print("=" * 60)
print("CASME II POOLFORMER BASELINE INFRASTRUCTURE")
print("=" * 60)
print("\n[1] Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully")

print("\n[2] Importing required libraries...")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import json
import os
import numpy as np
import pandas as pd
from PIL import Image
import time
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Project paths configuration - preserved existing structure
PROJECT_ROOT = "/content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project"
DATASET_ROOT = f"{PROJECT_ROOT}/datasets/processed_casme2/data_split_v1"
CHECKPOINT_ROOT = f"{PROJECT_ROOT}/models/02_05_poolformer_casme2-af"
RESULTS_ROOT = f"{PROJECT_ROOT}/results/02_05_poolformer_casme2-af"

# Load CASME II dataset metadata - preserved existing paths
METADATA_TRAIN = f"{DATASET_ROOT}/split_metadata.json"
PROCESSING_SUMMARY = f"{DATASET_ROOT}/processing_summary.json"

print("CASME II PoolFormer Baseline - Infrastructure Configuration")
print("=========================================================")

# Load dataset metadata
print("Loading CASME II dataset metadata...")
with open(METADATA_TRAIN, 'r') as f:
    casme2_metadata = json.load(f)

with open(PROCESSING_SUMMARY, 'r') as f:
    processing_info = json.load(f)

print(f"Dataset: {processing_info['dataset']}")
print(f"Total samples: {processing_info['total_samples']}")
print(f"Split strategy: {processing_info['split_strategy']}")

# =====================================================
# ADVANCED EXPERIMENT CONFIGURATION - Optimized Parameters
# =====================================================

# FOCAL LOSS CONFIGURATION - Toggle and Advanced Parameters
USE_FOCAL_LOSS = False  # Set True to enable Focal Loss, False for CrossEntropy
FOCAL_LOSS_GAMMA = 2.0  # Focal loss focusing parameter (typically 1.0 - 3.0)

# OPTIMIZED CLASS WEIGHTS CONFIGURATION - Inverse Square Root Frequency Approach
# CASME II classes: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
# Train distribution: [99, 63, 32, 27, 25, 7, 2] - inverse sqrt frequency approach

# CrossEntropy Loss - Optimized inverse square root frequency weights
CROSSENTROPY_CLASS_WEIGHTS = [1.00, 1.25, 1.76, 1.91, 1.99, 3.76, 7.04]

# Focal Loss - Normalized per-class alpha values (sum = 1.0)
FOCAL_LOSS_ALPHA_WEIGHTS = [0.053, 0.067, 0.094, 0.102, 0.106, 0.201, 0.376]

# POOLFORMER MODEL CONFIGURATION - Support m36 and m48 variants
POOLFORMER_MODEL_VARIANT = 'm36'  # Options: 'm36' or 'm48'

# Dynamic PoolFormer model selection based on variant
if POOLFORMER_MODEL_VARIANT == 'm36':
    POOLFORMER_MODEL_NAME = 'sail/poolformer_m36'
    MODEL_PARAMS = '56M'
    print("Using PoolFormer-M36 for micro-expression analysis (56M parameters)")
elif POOLFORMER_MODEL_VARIANT == 'm48':
    POOLFORMER_MODEL_NAME = 'sail/poolformer_m48'
    MODEL_PARAMS = '73M'
    print("Using PoolFormer-M48 for enhanced micro-expression recognition (73M parameters)")
else:
    raise ValueError(f"Unsupported POOLFORMER_MODEL_VARIANT: {POOLFORMER_MODEL_VARIANT}")

# Display experiment configuration
print("\n" + "=" * 50)
print("POOLFORMER EXPERIMENT CONFIGURATION")
print("=" * 50)
print(f"Loss Function: {'Focal Loss' if USE_FOCAL_LOSS else 'CrossEntropy Loss'}")
if USE_FOCAL_LOSS:
    print(f"  Gamma: {FOCAL_LOSS_GAMMA}")
    print(f"  Alpha Weights (per-class): {FOCAL_LOSS_ALPHA_WEIGHTS}")
    print(f"  Alpha Sum Validation: {sum(FOCAL_LOSS_ALPHA_WEIGHTS):.3f}")
else:
    print(f"  Class Weights (inverse sqrt freq): {CROSSENTROPY_CLASS_WEIGHTS}")
print(f"PoolFormer Model: {POOLFORMER_MODEL_NAME}")
print(f"Model Parameters: {MODEL_PARAMS}")
print("=" * 50)

# Enhanced GPU configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0

print(f"\nDevice: {device}")
print(f"GPU: {gpu_name} ({gpu_memory:.1f} GB)")

# Hardware-optimized batch size for 384px input
if 'A100' in gpu_name:
    BATCH_SIZE = 20
    NUM_WORKERS = 8
    torch.backends.cudnn.benchmark = True
    print("A100: Optimized batch size for PoolFormer 384px")
elif 'L4' in gpu_name:
    BATCH_SIZE = 16
    NUM_WORKERS = 6
    torch.backends.cudnn.benchmark = True
    print("L4: Balanced performance configuration")
else:
    BATCH_SIZE = 12
    NUM_WORKERS = 4
    print("Default GPU: Conservative settings")

# CASME II class mapping and analysis - preserved existing structure
CASME2_CLASSES = ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
CLASS_TO_IDX = {cls: idx for idx, cls in enumerate(CASME2_CLASSES)}

# Analyze class distribution from metadata
train_dist = casme2_metadata['train']['class_distribution']
val_dist = casme2_metadata['val']['class_distribution']
test_dist = casme2_metadata['test']['class_distribution']

print(f"\nTrain distribution: {train_dist}")
print(f"Validation distribution: {val_dist}")
print(f"Test distribution: {test_dist}")

# Apply optimized class weights based on loss function selection
if USE_FOCAL_LOSS:
    # For Focal Loss - use normalized alpha weights (per-class importance)
    class_weights = torch.tensor(FOCAL_LOSS_ALPHA_WEIGHTS, dtype=torch.float32).to(device)
    print(f"Applied Focal Loss alpha weights: {class_weights.cpu().numpy()}")
    print(f"Alpha weights sum: {class_weights.sum().item():.3f}")
else:
    # For CrossEntropy - use inverse sqrt frequency weights
    class_weights = torch.tensor(CROSSENTROPY_CLASS_WEIGHTS, dtype=torch.float32).to(device)
    print(f"Applied CrossEntropy class weights: {class_weights.cpu().numpy()}")

# CASME II PoolFormer Configuration
CASME2_POOLFORMER_CONFIG = {
    # Architecture configuration - enhanced flexibility
    'poolformer_model': POOLFORMER_MODEL_NAME,
    'model_variant': POOLFORMER_MODEL_VARIANT,
    'model_params': MODEL_PARAMS,
    'input_size': 384,
    'num_classes': 7,
    'dropout_rate': 0.2,
    'expected_feature_dim': 768,

    # Training configuration (proven effective from medical imaging)
    'learning_rate': 1e-5,
    'weight_decay': 1e-5,
    'gradient_clip': 1.0,
    'num_epochs': 50,
    'batch_size': BATCH_SIZE,
    'num_workers': NUM_WORKERS,
    'device': device,

    # Scheduler configuration
    'scheduler_type': 'plateau',
    'scheduler_mode': 'max',
    'scheduler_factor': 0.5,
    'scheduler_patience': 3,
    'scheduler_min_lr': 1e-6,
    'scheduler_monitor': 'val_f1_macro',

    # Optimized loss configuration
    'use_focal_loss': USE_FOCAL_LOSS,
    'focal_loss_gamma': FOCAL_LOSS_GAMMA,
    'focal_loss_alpha_weights': FOCAL_LOSS_ALPHA_WEIGHTS,
    'crossentropy_class_weights': CROSSENTROPY_CLASS_WEIGHTS,
    'class_weights': class_weights,

    # Evaluation configuration
    'use_macro_avg': True,
    'early_stopping': False,
    'save_best_f1': True,
    'save_strategy': 'best_only'
}

print(f"\nPoolFormer Configuration Summary:")
print(f"  Model: {CASME2_POOLFORMER_CONFIG['poolformer_model']}")
print(f"  Input size: {CASME2_POOLFORMER_CONFIG['input_size']}px")
print(f"  Learning rate: {CASME2_POOLFORMER_CONFIG['learning_rate']}")
print(f"  Batch size: {BATCH_SIZE}")

# =====================================================
# ADVANCED FOCAL LOSS IMPLEMENTATION - Per-Class Alpha Support
# =====================================================

class OptimizedFocalLoss(nn.Module):
    """
    Advanced Focal Loss implementation with per-class alpha support
    Paper: "Focal Loss for Dense Object Detection" (Lin et al., 2017)

    Enhanced Formula: FL(p_t) = -α_t(1-p_t)^γ log(p_t)

    Args:
        alpha (list/tensor): Per-class alpha weights (must sum to 1.0)
        gamma (float): Focusing parameter for hard examples (default: 2.0)
        reduction (str): Reduction method ('mean', 'sum', 'none')
    """

    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(OptimizedFocalLoss, self).__init__()

        if alpha is not None:
            if isinstance(alpha, list):
                self.alpha = torch.tensor(alpha, dtype=torch.float32)
            else:
                self.alpha = alpha

            # Validation: alpha should sum to 1.0 for proper normalization
            alpha_sum = self.alpha.sum().item()
            if abs(alpha_sum - 1.0) > 0.01:
                print(f"Warning: Alpha weights sum to {alpha_sum:.3f}, expected 1.0")

        else:
            self.alpha = None

        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Calculate cross entropy loss
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        # Calculate p_t (probability of true class)
        pt = torch.exp(-ce_loss)

        # Apply per-class alpha if provided
        if self.alpha is not None:
            if self.alpha.device != targets.device:
                self.alpha = self.alpha.to(targets.device)
            alpha_t = self.alpha.gather(0, targets)
        else:
            alpha_t = 1.0

        # Apply focal loss formula: α_t(1-p_t)^γ * CE_loss
        focal_loss = alpha_t * (1 - pt) ** self.gamma * ce_loss

        # Apply reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# PoolFormer Architecture for CASME II
class PoolFormerCASME2Baseline(nn.Module):
    """PoolFormer baseline for CASME II micro-expression recognition with Hugging Face transformers"""

    def __init__(self, num_classes, dropout_rate=0.2):
        super(PoolFormerCASME2Baseline, self).__init__()

        # Hugging Face PoolFormer model
        from transformers import PoolFormerModel

        self.poolformer = PoolFormerModel.from_pretrained(
            CASME2_POOLFORMER_CONFIG['poolformer_model']
        )

        # Enable fine-tuning for micro-expression domain
        for param in self.poolformer.parameters():
            param.requires_grad = True

        # Get PoolFormer feature dimensions
        self.poolformer_feature_dim = self.poolformer.config.hidden_sizes[-1]

        print(f"PoolFormer feature dimension: {self.poolformer_feature_dim}")

        # Classification head with LayerNorm for stability
        self.classifier_layers = nn.Sequential(
            nn.Linear(self.poolformer_feature_dim, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(dropout_rate),

            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(dropout_rate),
        )

        # Final classification layer for CASME II classes
        self.classifier = nn.Linear(128, num_classes)

        print(f"PoolFormer CASME II: {self.poolformer_feature_dim} -> 512 -> 128 -> {num_classes}")

    def forward(self, pixel_values):
        # PoolFormer forward pass
        poolformer_outputs = self.poolformer(pixel_values=pixel_values)

        # Extract pooled features from PoolFormer last hidden state
        # PoolFormer output: [batch_size, channels, height, width]
        poolformer_features = poolformer_outputs.last_hidden_state

        # Global average pooling across spatial dimensions for classification
        # [batch_size, channels, height, width] -> [batch_size, channels]
        poolformer_features = poolformer_features.mean(dim=[2, 3])

        # Classification pipeline
        processed_features = self.classifier_layers(poolformer_features)
        output = self.classifier(processed_features)

        return output

# Enhanced optimizer and scheduler factory
def create_optimizer_scheduler_casme2(model, config):
    """Create optimizer and scheduler for CASME II PoolFormer training"""

    # AdamW optimizer with proven configuration
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay'],
        betas=(0.9, 0.999)
    )

    # ReduceLROnPlateau scheduler monitoring validation F1
    if config['scheduler_type'] == 'plateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=config['scheduler_mode'],
            factor=config['scheduler_factor'],
            patience=config['scheduler_patience'],
            min_lr=config['scheduler_min_lr']
        )
        print(f"Scheduler: ReduceLROnPlateau monitoring {config['scheduler_monitor']}")
    else:
        scheduler = None

    return optimizer, scheduler

# PoolFormer Image Processor setup for 384px input
from transformers import PoolFormerImageProcessor

print("\nSetting up PoolFormer Image Processor for 384px input...")

poolformer_processor = PoolFormerImageProcessor.from_pretrained(
    CASME2_POOLFORMER_CONFIG['poolformer_model'],
    do_resize=True,
    size={'height': 384, 'width': 384},
    do_normalize=True,
    do_rescale=True,
    do_center_crop=False
)

# Transform functions for PoolFormer
def poolformer_transform_train(image):
    """Training transform with PoolFormer Image Processor"""
    inputs = poolformer_processor(image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

def poolformer_transform_val(image):
    """Validation transform with PoolFormer Image Processor"""
    inputs = poolformer_processor(image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

print("PoolFormer Image Processor configured for 384px input")

# Custom Dataset class for CASME II - preserved existing structure
class CASME2Dataset(Dataset):
    """Custom dataset class for CASME II with JSON metadata support"""

    def __init__(self, split_metadata, dataset_root, transform=None, split='train'):
        self.metadata = split_metadata[split]['samples']
        self.dataset_root = dataset_root
        self.transform = transform
        self.split = split

        print(f"Loaded {len(self.metadata)} samples for {split} split")

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        sample = self.metadata[idx]

        # Load image
        image_path = os.path.join(self.dataset_root, self.split, sample['image_filename'])
        image = Image.open(image_path).convert('RGB')

        # Apply transform
        if self.transform:
            image = self.transform(image)

        # Get class label
        emotion = sample['emotion']
        label = CLASS_TO_IDX[emotion]

        return image, label, sample['sample_id']

# Create directories - preserved existing paths
os.makedirs(CHECKPOINT_ROOT, exist_ok=True)
os.makedirs(f"{RESULTS_ROOT}/training_logs", exist_ok=True)
os.makedirs(f"{RESULTS_ROOT}/evaluation_results", exist_ok=True)

# Dataset paths - preserved existing structure
TRAIN_PATH = f"{DATASET_ROOT}/train"
VAL_PATH = f"{DATASET_ROOT}/val"
TEST_PATH = f"{DATASET_ROOT}/test"

print(f"\nDataset paths:")
print(f"Train: {TRAIN_PATH}")
print(f"Validation: {VAL_PATH}")
print(f"Test: {TEST_PATH}")

# Enhanced architecture validation with dynamic feature calculation
print("\nPoolFormer CASME II architecture validation...")

try:
    test_model = PoolFormerCASME2Baseline(num_classes=7, dropout_rate=0.2).to(device)
    test_input = torch.randn(1, 3, 384, 384).to(device)
    test_output = test_model(test_input)

    print(f"Validation successful: Output shape {test_output.shape}")
    print(f"PoolFormer {POOLFORMER_MODEL_VARIANT.upper()} with {MODEL_PARAMS} parameters")
    print(f"Input resolution: 384x384px")

    del test_model, test_input, test_output
    torch.cuda.empty_cache()

except Exception as e:
    print(f"Validation failed: {e}")

# Optimized loss function factory with advanced configuration
def create_criterion_casme2(weights, use_focal_loss=False, alpha_weights=None, gamma=2.0):
    """
    Optimized factory function to create loss criterion based on advanced configuration

    Args:
        weights (Tensor): Class weights for CrossEntropy (ignored if focal loss used)
        use_focal_loss (bool): Whether to use Focal Loss or CrossEntropy
        alpha_weights (list): Per-class alpha weights for Focal Loss (must sum to 1.0)
        gamma (float): Focal loss gamma parameter

    Returns:
        Loss function (nn.Module)
    """
    if use_focal_loss:
        print(f"Using Optimized Focal Loss with gamma={gamma}")
        if alpha_weights:
            print(f"Per-class alpha weights: {alpha_weights}")
            print(f"Alpha sum: {sum(alpha_weights):.3f}")
        return OptimizedFocalLoss(alpha=alpha_weights, gamma=gamma)
    else:
        print(f"Using CrossEntropy Loss with optimized class weights")
        print(f"Class weights: {weights.cpu().numpy()}")
        return nn.CrossEntropyLoss(weight=weights)

# Global configuration for training pipeline - enhanced with optimized features
GLOBAL_CONFIG_CASME2 = {
    'device': device,
    'batch_size': BATCH_SIZE,
    'num_workers': NUM_WORKERS,
    'num_classes': 7,
    'class_weights': class_weights,
    'class_names': CASME2_CLASSES,
    'class_to_idx': CLASS_TO_IDX,
    'transform_train': poolformer_transform_train,
    'transform_val': poolformer_transform_val,
    'poolformer_config': CASME2_POOLFORMER_CONFIG,
    'checkpoint_root': CHECKPOINT_ROOT,
    'results_root': RESULTS_ROOT,
    'train_path': TRAIN_PATH,
    'val_path': VAL_PATH,
    'test_path': TEST_PATH,
    'metadata': casme2_metadata,
    'optimizer_scheduler_factory': create_optimizer_scheduler_casme2,
    'criterion_factory': create_criterion_casme2
}

# Configuration validation and summary
print("\n" + "=" * 60)
print("CASME II POOLFORMER BASELINE CONFIGURATION COMPLETE")
print("=" * 60)

print(f"Loss Configuration:")
if USE_FOCAL_LOSS:
    print(f"  Function: Optimized Focal Loss")
    print(f"  Gamma: {FOCAL_LOSS_GAMMA}")
    print(f"  Per-class Alpha: {FOCAL_LOSS_ALPHA_WEIGHTS}")
    print(f"  Alpha Sum: {sum(FOCAL_LOSS_ALPHA_WEIGHTS):.3f}")
else:
    print(f"  Function: CrossEntropy with Optimized Weights")
    print(f"  Class Weights: {CROSSENTROPY_CLASS_WEIGHTS}")

print(f"\nModel Configuration:")
print(f"  Architecture: {POOLFORMER_MODEL_NAME}")
print(f"  Model Variant: {POOLFORMER_MODEL_VARIANT.upper()}")
print(f"  Parameters: {MODEL_PARAMS}")
print(f"  Input Resolution: 224px")

print(f"\nDataset Configuration:")
print(f"  Classes: {len(CASME2_CLASSES)}")
print(f"  Weight Optimization: {'Per-class Alpha' if USE_FOCAL_LOSS else 'Inverse Sqrt Frequency'}")

print("\nNext: Cell 2 - Dataset Loading and Optimized Training Pipeline")

CASME II POOLFORMER BASELINE INFRASTRUCTURE

[1] Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully

[2] Importing required libraries...
CASME II PoolFormer Baseline - Infrastructure Configuration
Loading CASME II dataset metadata...
Dataset: CASME2
Total samples: 255
Split strategy: stratified_80_10_10
Using PoolFormer-M48 for enhanced micro-expression recognition (73M parameters)

POOLFORMER EXPERIMENT CONFIGURATION
Loss Function: Focal Loss
  Gamma: 2.0
  Alpha Weights (per-class): [0.053, 0.067, 0.094, 0.102, 0.106, 0.201, 0.376]
  Alpha Sum Validation: 0.999
PoolFormer Model: sail/poolformer_m48
Model Parameters: 73M

Device: cuda
GPU: NVIDIA L4 (23.8 GB)
L4: Balanced performance configuration

Train distribution: {'others': 79, 'disgust': 50, 'happiness': 25, 'repression': 21, 'surprise': 20, 'sadness': 5, 'fear': 1}
Validation distribution: {'others': 10, 'disgust': 6, 'happiness': 3, 'repression': 3, 'surprise': 2, 'sadness': 1, 'fear': 1}
Test

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

PoolFormer Image Processor configured for 384px input

Dataset paths:
Train: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/data_split/train
Validation: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/data_split/val
Test: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/data_split/test

PoolFormer CASME II architecture validation...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/294M [00:00<?, ?B/s]

PoolFormer feature dimension: 768
PoolFormer CASME II: 768 -> 512 -> 128 -> 7
Validation successful: Output shape torch.Size([1, 7])
PoolFormer M48 with 73M parameters
Input resolution: 384x384px

CASME II POOLFORMER BASELINE CONFIGURATION COMPLETE
Loss Configuration:
  Function: Optimized Focal Loss
  Gamma: 2.0
  Per-class Alpha: [0.053, 0.067, 0.094, 0.102, 0.106, 0.201, 0.376]
  Alpha Sum: 0.999

Model Configuration:
  Architecture: sail/poolformer_m48
  Model Variant: M48
  Parameters: 73M
  Input Resolution: 224px

Dataset Configuration:
  Classes: 7
  Weight Optimization: Per-class Alpha

Next: Cell 2 - Dataset Loading and Optimized Training Pipeline


In [ ]:
# @title Cell 2: CASME II PoolFormer Training Pipeline

# File: 02_05_PoolFormer_CASME2_AF_Cell2.py
# Location: experiments/02_05_PoolFormer_CASME2-AF.ipynb
# Purpose: Enhanced training pipeline for CASME II PoolFormer micro-expression recognition with simplified checkpoint saving

import os
import time
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

print("CASME II PoolFormer Enhanced Training Pipeline with Configurable Loss")
print("=" * 65)
print(f"Loss Function: {'Optimized Focal Loss' if CASME2_POOLFORMER_CONFIG['use_focal_loss'] else 'CrossEntropy Loss'}")
if CASME2_POOLFORMER_CONFIG['use_focal_loss']:
    print(f"Focal Loss Parameters:")
    print(f"  Gamma: {CASME2_POOLFORMER_CONFIG['focal_loss_gamma']}")
    print(f"  Per-class Alpha: {CASME2_POOLFORMER_CONFIG['focal_loss_alpha_weights']}")
    print(f"  Alpha Sum: {sum(CASME2_POOLFORMER_CONFIG['focal_loss_alpha_weights']):.3f}")
else:
    print(f"CrossEntropy Parameters:")
    print(f"  Optimized Class Weights: {CASME2_POOLFORMER_CONFIG['crossentropy_class_weights']}")
print(f"Training epochs: {CASME2_POOLFORMER_CONFIG['num_epochs']}")
print(f"Scheduler patience: {CASME2_POOLFORMER_CONFIG['scheduler_patience']}")
print(f"Early stopping: {CASME2_POOLFORMER_CONFIG['early_stopping']}")

# Enhanced CASME II Dataset with optimized RAM caching - preserved existing structure
class CASME2DatasetTraining(Dataset):
    """Enhanced CASME II dataset for training with RAM caching optimization"""

    def __init__(self, split_metadata, dataset_root, transform=None, split='train', use_ram_cache=True):
        self.metadata = split_metadata[split]['samples']
        self.dataset_root = dataset_root
        self.transform = transform
        self.split = split
        self.use_ram_cache = use_ram_cache
        self.images = []
        self.labels = []
        self.sample_ids = []
        self.cached_images = []

        print(f"Loading CASME II {split} dataset for training...")

        # Process metadata
        for sample in self.metadata:
            image_path = os.path.join(dataset_root, split, sample['image_filename'])
            self.images.append(image_path)
            self.labels.append(CLASS_TO_IDX[sample['emotion']])
            self.sample_ids.append(sample['sample_id'])

        print(f"Loaded {len(self.images)} CASME II {split} samples")
        self._print_distribution()

        # RAM caching for training efficiency
        if self.use_ram_cache:
            self._preload_to_ram()

    def _print_distribution(self):
        """Print class distribution"""
        label_counts = {}
        for label in self.labels:
            label_counts[label] = label_counts.get(label, 0) + 1

        for label, count in sorted(label_counts.items()):
            class_name = CASME2_CLASSES[label]
            percentage = (count / len(self.labels)) * 100
            print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

    def _preload_to_ram(self):
        """RAM preloading for training efficiency"""
        print(f"Preloading {len(self.images)} {self.split} images to RAM...")

        max_workers = min(mp.cpu_count(), 8)
        valid_images = 0
        self.cached_images = [None] * len(self.images)

        for i, img_path in enumerate(tqdm(self.images, desc=f"Loading {self.split}")):
            try:
                image = Image.open(img_path).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
                self.cached_images[i] = image
                valid_images += 1
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                self.cached_images[i] = Image.new('RGB', (224, 224), (128, 128, 128))

        ram_usage_gb = len(self.cached_images) * 224 * 224 * 3 * 4 / 1e9
        print(f"{self.split.upper()} RAM caching completed: {valid_images} images, ~{ram_usage_gb:.2f}GB")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if self.use_ram_cache and self.cached_images[idx] is not None:
            image = self.cached_images[idx].copy()
        else:
            try:
                image = Image.open(self.images[idx]).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
            except:
                image = Image.new('RGB', (224, 224), (128, 128, 128))

        if self.transform:
            image = self.transform(image)

        return image, self.labels[idx], self.sample_ids[idx]

# Enhanced metrics calculation with comprehensive error handling
def calculate_metrics_safe_robust(outputs, labels, class_names, average='macro'):
    """Calculate metrics with enhanced error handling and validation"""
    try:
        # Validate input tensors
        if outputs.size(0) != labels.size(0):
            raise ValueError(f"Batch size mismatch: outputs {outputs.size(0)} vs labels {labels.size(0)}")

        if isinstance(outputs, torch.Tensor):
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()
        else:
            predictions = np.array(outputs)

        if isinstance(labels, torch.Tensor):
            labels = labels.cpu().numpy()
        else:
            labels = np.array(labels)

        # Validate predictions are in valid range
        unique_preds = np.unique(predictions)
        unique_labels = np.unique(labels)

        accuracy = accuracy_score(labels, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions,
            average=average,
            zero_division=0,
            labels=list(range(len(class_names)))
        )

        return {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1)
        }
    except Exception as e:
        print(f"Warning: Enhanced metrics calculation error: {e}")
        return {
            'accuracy': 0.0,
            'precision': 0.0,
            'recall': 0.0,
            'f1_score': 0.0
        }

# Enhanced training epoch function with robust model output validation
def train_epoch(model, dataloader, criterion, optimizer, device, epoch, total_epochs):
    """Enhanced training epoch with robust error handling and output validation"""
    model.train()
    running_loss = 0.0
    all_outputs = []
    all_labels = []

    progress_bar = tqdm(dataloader, desc=f"CASME II Training Epoch {epoch+1}/{total_epochs}")

    for batch_idx, (images, labels, sample_ids) in enumerate(progress_bar):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Enhanced model output validation - handle multiple output formats
        model_output = model(images)

        # Robust output structure validation
        if isinstance(model_output, (tuple, list)):
            outputs = model_output[0]
        elif isinstance(model_output, dict):
            outputs = model_output.get('logits', model_output.get('prediction', model_output))
        else:
            outputs = model_output

        # Validate output shape for 7 CASME II classes
        if outputs.dim() != 2 or outputs.size(1) != 7:
            raise ValueError(f"Invalid CASME II output shape: {outputs.shape}, expected [batch_size, 7]")

        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), CASME2_POOLFORMER_CONFIG['gradient_clip'])

        optimizer.step()
        running_loss += loss.item()

        # Memory optimized: Move to CPU before accumulating
        all_outputs.append(outputs.detach().cpu())
        all_labels.append(labels.detach().cpu())

        # Update progress
        if batch_idx % 5 == 0:
            avg_loss = running_loss / (batch_idx + 1)
            current_lr = optimizer.param_groups[0]['lr']
            progress_bar.set_postfix({
                'Loss': f'{avg_loss:.4f}',
                'LR': f'{current_lr:.2e}'
            })

    # Enhanced metrics calculation with error recovery
    try:
        epoch_outputs = torch.cat(all_outputs, dim=0)
        epoch_labels = torch.cat(all_labels, dim=0)
        metrics = calculate_metrics_safe_robust(epoch_outputs, epoch_labels, CASME2_CLASSES, average='macro')
    except Exception as e:
        print(f"Warning: Training metrics calculation failed: {e}")
        metrics = {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

    avg_loss = running_loss / len(dataloader)
    return avg_loss, metrics

# Enhanced validation epoch function with robust model output validation
def validate_epoch(model, dataloader, criterion, device, epoch, total_epochs):
    """Enhanced validation epoch with robust error handling and output validation"""
    model.eval()
    running_loss = 0.0
    all_outputs = []
    all_labels = []
    all_sample_ids = []

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc=f"CASME II Validation Epoch {epoch+1}/{total_epochs}")

        for batch_idx, (images, labels, sample_ids) in enumerate(progress_bar):
            images, labels = images.to(device), labels.to(device)

            # Enhanced model output validation - handle multiple output formats
            model_output = model(images)

            # Robust output structure validation
            if isinstance(model_output, (tuple, list)):
                outputs = model_output[0]
            elif isinstance(model_output, dict):
                outputs = model_output.get('logits', model_output.get('prediction', model_output))
            else:
                outputs = model_output

            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Memory optimized: Move to CPU before accumulating
            all_outputs.append(outputs.detach().cpu())
            all_labels.append(labels.detach().cpu())
            all_sample_ids.extend(sample_ids)

            if batch_idx % 3 == 0:
                avg_loss = running_loss / (batch_idx + 1)
                progress_bar.set_postfix({'Val Loss': f'{avg_loss:.4f}'})

    # Enhanced metrics calculation with error recovery
    try:
        epoch_outputs = torch.cat(all_outputs, dim=0)
        epoch_labels = torch.cat(all_labels, dim=0)
        metrics = calculate_metrics_safe_robust(epoch_outputs, epoch_labels, CASME2_CLASSES, average='macro')
    except Exception as e:
        print(f"Warning: Validation metrics calculation failed: {e}")
        metrics = {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

    avg_loss = running_loss / len(dataloader)
    return avg_loss, metrics, all_sample_ids

# Simplified checkpoint saving function with single best model approach
def save_checkpoint_simplified(model, optimizer, scheduler, epoch, train_metrics, val_metrics,
                             checkpoint_dir, best_metrics, config):
    """Simplified checkpoint saving - only saves best F1 model"""

    # Convert all tensors to serializable format
    def make_serializable(obj):
        if isinstance(obj, torch.Tensor):
            return obj.cpu().item() if obj.numel() == 1 else obj.cpu().tolist()
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, dict):
            return {k: make_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [make_serializable(item) for item in obj]
        else:
            return obj

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'train_metrics': make_serializable(train_metrics),
        'val_metrics': make_serializable(val_metrics),
        'casme2_config': make_serializable(config),
        'best_f1': float(best_metrics['f1']),
        'best_loss': float(best_metrics['loss']),
        'best_acc': float(best_metrics['accuracy']),
        'class_names': CASME2_CLASSES,
        'num_classes': 7
    }

    best_path = f"{checkpoint_dir}/casme2_poolformer_best_f1.pth"

    try:
        torch.save(checkpoint, best_path)
        print(f"Best model checkpoint saved: {os.path.basename(best_path)}")
        return best_path
    except Exception as e:
        print(f"Error saving checkpoint: {e}")
        return None

# Safe JSON serialization function - preserved existing function
def safe_json_serialize(obj):
    """Convert objects to JSON-serializable format"""
    if isinstance(obj, torch.Tensor):
        return obj.cpu().item() if obj.numel() == 1 else obj.cpu().numpy().tolist()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.integer, np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: safe_json_serialize(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [safe_json_serialize(item) for item in obj]
    elif hasattr(obj, '__dict__'):
        return safe_json_serialize(obj.__dict__)
    else:
        try:
            # Try to convert to basic Python types
            return float(obj) if isinstance(obj, (int, float)) else str(obj)
        except:
            return str(obj)

# Create enhanced datasets - preserved existing structure
print("\nCreating CASME II training datasets...")

train_dataset = CASME2DatasetTraining(
    split_metadata=GLOBAL_CONFIG_CASME2['metadata'],
    dataset_root=GLOBAL_CONFIG_CASME2['train_path'].replace('/train', ''),
    transform=GLOBAL_CONFIG_CASME2['transform_train'],
    split='train',
    use_ram_cache=True
)

val_dataset = CASME2DatasetTraining(
    split_metadata=GLOBAL_CONFIG_CASME2['metadata'],
    dataset_root=GLOBAL_CONFIG_CASME2['val_path'].replace('/val', ''),
    transform=GLOBAL_CONFIG_CASME2['transform_val'],
    split='val',
    use_ram_cache=True
)

# Create data loaders - preserved existing structure
train_loader = DataLoader(
    train_dataset,
    batch_size=CASME2_POOLFORMER_CONFIG['batch_size'],
    shuffle=True,
    num_workers=CASME2_POOLFORMER_CONFIG['num_workers'],
    pin_memory=True,
    prefetch_factor=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CASME2_POOLFORMER_CONFIG['batch_size'],
    shuffle=False,
    num_workers=CASME2_POOLFORMER_CONFIG['num_workers'],
    pin_memory=True,
    prefetch_factor=2
)

print(f"Training batches: {len(train_loader)} (samples: {len(train_dataset)})")
print(f"Validation batches: {len(val_loader)} (samples: {len(val_dataset)})")

# Initialize model, criterion, optimizer, scheduler
print("\nInitializing CASME II PoolFormer enhanced model...")
model = PoolFormerCASME2Baseline(
    num_classes=GLOBAL_CONFIG_CASME2['num_classes'],
    dropout_rate=CASME2_POOLFORMER_CONFIG['dropout_rate']
).to(GLOBAL_CONFIG_CASME2['device'])

# Enhanced criterion creation using configurable factory function
if CASME2_POOLFORMER_CONFIG['use_focal_loss']:
    criterion = GLOBAL_CONFIG_CASME2['criterion_factory'](
        weights=GLOBAL_CONFIG_CASME2['class_weights'],
        use_focal_loss=True,
        alpha_weights=CASME2_POOLFORMER_CONFIG['focal_loss_alpha_weights'],
        gamma=CASME2_POOLFORMER_CONFIG['focal_loss_gamma']
    )
else:
    criterion = GLOBAL_CONFIG_CASME2['criterion_factory'](
        weights=GLOBAL_CONFIG_CASME2['class_weights'],
        use_focal_loss=False,
        alpha_weights=None,
        gamma=2.0
    )

optimizer, scheduler = GLOBAL_CONFIG_CASME2['optimizer_scheduler_factory'](
    model, CASME2_POOLFORMER_CONFIG
)

print(f"Optimizer: AdamW (LR={CASME2_POOLFORMER_CONFIG['learning_rate']})")
print(f"Scheduler: ReduceLROnPlateau (patience={CASME2_POOLFORMER_CONFIG['scheduler_patience']})")
print(f"Criterion: {'Optimized Focal Loss' if CASME2_POOLFORMER_CONFIG['use_focal_loss'] else 'CrossEntropy'}")

# Training history tracking - preserved existing structure
training_history = {
    'train_loss': [],
    'val_loss': [],
    'train_f1': [],
    'val_f1': [],
    'train_acc': [],
    'val_acc': [],
    'learning_rate': [],
    'epoch_time': []
}

# Simplified best metrics tracking for single checkpoint
best_metrics = {
    'f1': 0.0,
    'loss': float('inf'),
    'accuracy': 0.0,
    'epoch': 0
}

print("\nStarting CASME II PoolFormer enhanced training...")
print(f"Training configuration: {CASME2_POOLFORMER_CONFIG['num_epochs']} epochs")
print("=" * 65)

# Main training loop with simplified checkpoint logic
start_time = time.time()

for epoch in range(CASME2_POOLFORMER_CONFIG['num_epochs']):
    epoch_start_time = time.time()
    print(f"\nEpoch {epoch+1}/{CASME2_POOLFORMER_CONFIG['num_epochs']}")

    # Training phase
    train_loss, train_metrics = train_epoch(
        model, train_loader, criterion, optimizer,
        GLOBAL_CONFIG_CASME2['device'], epoch, CASME2_POOLFORMER_CONFIG['num_epochs']
    )

    # Validation phase
    val_loss, val_metrics, val_sample_ids = validate_epoch(
        model, val_loader, criterion,
        GLOBAL_CONFIG_CASME2['device'], epoch, CASME2_POOLFORMER_CONFIG['num_epochs']
    )

    # Update scheduler
    if scheduler:
        scheduler.step(val_metrics['f1_score'])

    # Record training history
    epoch_time = time.time() - epoch_start_time
    current_lr = optimizer.param_groups[0]['lr']

    training_history['train_loss'].append(float(train_loss))
    training_history['val_loss'].append(float(val_loss))
    training_history['train_f1'].append(float(train_metrics['f1_score']))
    training_history['val_f1'].append(float(val_metrics['f1_score']))
    training_history['train_acc'].append(float(train_metrics['accuracy']))
    training_history['val_acc'].append(float(val_metrics['accuracy']))
    training_history['learning_rate'].append(float(current_lr))
    training_history['epoch_time'].append(float(epoch_time))

    # Print epoch summary
    print(f"Train - Loss: {train_loss:.4f}, F1: {train_metrics['f1_score']:.4f}, Acc: {train_metrics['accuracy']:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, F1: {val_metrics['f1_score']:.4f}, Acc: {val_metrics['accuracy']:.4f}")
    print(f"Time  - Epoch: {epoch_time:.1f}s, LR: {current_lr:.2e}")

    # Simplified checkpoint saving - only save if F1 improves
    if val_metrics['f1_score'] > best_metrics['f1']:
        best_metrics['f1'] = val_metrics['f1_score']
        best_metrics['loss'] = val_loss
        best_metrics['accuracy'] = val_metrics['accuracy']
        best_metrics['epoch'] = epoch + 1

        saved_path = save_checkpoint_simplified(
            model, optimizer, scheduler, epoch,
            train_metrics, val_metrics, GLOBAL_CONFIG_CASME2['checkpoint_root'],
            best_metrics, CASME2_POOLFORMER_CONFIG
        )

        if saved_path:
            print(f"New best model saved - F1: {best_metrics['f1']:.4f}")

    # Progress tracking
    elapsed_time = time.time() - start_time
    estimated_total = (elapsed_time / (epoch + 1)) * CASME2_POOLFORMER_CONFIG['num_epochs']
    remaining_time = estimated_total - elapsed_time
    progress_pct = ((epoch + 1) / CASME2_POOLFORMER_CONFIG['num_epochs']) * 100

    print(f"Progress: {progress_pct:.1f}% | Best F1: {best_metrics['f1']:.4f} | ETA: {remaining_time/60:.1f}min")

# Training completion
total_time = time.time() - start_time
actual_epochs = CASME2_POOLFORMER_CONFIG['num_epochs']

print("\n" + "=" * 65)
print("CASME II POOLFORMER BASELINE TRAINING COMPLETED")
print("=" * 65)
print(f"Training time: {total_time/60:.1f} minutes")
print(f"Epochs completed: {actual_epochs}")
print(f"Best validation F1: {best_metrics['f1']:.4f} (epoch {best_metrics['epoch']})")
print(f"Final train F1: {training_history['train_f1'][-1]:.4f}")
print(f"Final validation F1: {training_history['val_f1'][-1]:.4f}")

# Enhanced training documentation export
results_dir = GLOBAL_CONFIG_CASME2['results_root']
os.makedirs(f"{results_dir}/training_logs", exist_ok=True)

training_history_path = f"{results_dir}/training_logs/casme2_poolformer_training_history.json"

print("\nExporting enhanced training documentation...")

try:
    # Create comprehensive training summary with enhanced experiment configuration
    training_summary = {
        'experiment_type': 'CASME2_PoolFormer_Baseline',
        'experiment_configuration': {
            'loss_function': 'Optimized Focal Loss' if CASME2_POOLFORMER_CONFIG['use_focal_loss'] else 'CrossEntropy',
            'weight_approach': 'Per-class Alpha (sum=1.0)' if CASME2_POOLFORMER_CONFIG['use_focal_loss'] else 'Inverse Sqrt Frequency',
            'focal_loss_gamma': CASME2_POOLFORMER_CONFIG['focal_loss_gamma'],
            'focal_loss_alpha_weights': CASME2_POOLFORMER_CONFIG['focal_loss_alpha_weights'],
            'crossentropy_class_weights': CASME2_POOLFORMER_CONFIG['crossentropy_class_weights'],
            'poolformer_model': CASME2_POOLFORMER_CONFIG['poolformer_model'],
            'model_variant': CASME2_POOLFORMER_CONFIG['model_variant'],
            'model_params': CASME2_POOLFORMER_CONFIG['model_params']
        },
        'training_history': safe_json_serialize(training_history),
        'best_val_f1': float(best_metrics['f1']),
        'best_val_loss': float(best_metrics['loss']),
        'best_val_accuracy': float(best_metrics['accuracy']),
        'best_epoch': int(best_metrics['epoch']),
        'total_epochs': int(actual_epochs),
        'total_time_minutes': float(total_time / 60),
        'average_epoch_time_seconds': float(np.mean(training_history['epoch_time'])),
        'config': safe_json_serialize(CASME2_POOLFORMER_CONFIG),
        'final_train_f1': float(training_history['train_f1'][-1]),
        'final_val_f1': float(training_history['val_f1'][-1]),
        'model_checkpoint': 'casme2_poolformer_best_f1.pth',
        'dataset_info': {
            'name': 'CASME_II',
            'total_samples': 255,
            'train_samples': len(train_dataset),
            'val_samples': len(val_dataset),
            'num_classes': 7,
            'class_names': CASME2_CLASSES
        },
        'architecture_info': {
            'model_type': 'PoolFormerCASME2Baseline',
            'backbone': CASME2_POOLFORMER_CONFIG['poolformer_model'],
            'variant': CASME2_POOLFORMER_CONFIG['model_variant'],
            'parameters': CASME2_POOLFORMER_CONFIG['model_params'],
            'input_size': f"{CASME2_POOLFORMER_CONFIG['input_size']}x{CASME2_POOLFORMER_CONFIG['input_size']}",
            'classification_head': 'pooled_features->512->128->7'
        },
        'enhanced_features': {
            'simplified_checkpoint_saving': True,
            'model_output_validation': True,
            'enhanced_error_handling': True,
            'single_best_model_logic': True,
            'memory_optimized_training': True,
            'poolformer_metaformer_architecture': True
        }
    }

    # Save with proper JSON serialization
    with open(training_history_path, 'w') as f:
        json.dump(training_summary, f, indent=2)

    print(f"Enhanced training documentation saved successfully: {training_history_path}")
    print(f"Experiment details: {training_summary['experiment_configuration']['loss_function']} loss")
    if CASME2_POOLFORMER_CONFIG['use_focal_loss']:
        print(f"  Gamma: {CASME2_POOLFORMER_CONFIG['focal_loss_gamma']}, Alpha Sum: {sum(CASME2_POOLFORMER_CONFIG['focal_loss_alpha_weights']):.3f}")
    print(f"Model variant: {CASME2_POOLFORMER_CONFIG['poolformer_model']} ({CASME2_POOLFORMER_CONFIG['model_params']})")

except Exception as e:
    print(f"Warning: Could not save training documentation: {e}")
    print("Training completed successfully, but documentation export failed")

# Enhanced memory cleanup
if torch.cuda.is_available():
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

print("\nNext: Cell 3 - CASME II PoolFormer Enhanced Evaluation")
print("Enhanced training pipeline completed successfully!")

CASME II PoolFormer Enhanced Training Pipeline with Configurable Loss
Loss Function: Optimized Focal Loss
Focal Loss Parameters:
  Gamma: 2.0
  Per-class Alpha: [0.053, 0.067, 0.094, 0.102, 0.106, 0.201, 0.376]
  Alpha Sum: 0.999
Training epochs: 50
Scheduler patience: 3
Early stopping: False

Creating CASME II training datasets...
Loading CASME II train dataset for training...
Loaded 201 CASME II train samples
  others: 79 samples (39.3%)
  disgust: 50 samples (24.9%)
  happiness: 25 samples (12.4%)
  repression: 21 samples (10.4%)
  surprise: 20 samples (10.0%)
  sadness: 5 samples (2.5%)
  fear: 1 samples (0.5%)
Preloading 201 train images to RAM...


Loading train: 100%|██████████| 201/201 [01:16<00:00,  2.62it/s]


TRAIN RAM caching completed: 201 images, ~0.12GB
Loading CASME II val dataset for training...
Loaded 26 CASME II val samples
  others: 10 samples (38.5%)
  disgust: 6 samples (23.1%)
  happiness: 3 samples (11.5%)
  repression: 3 samples (11.5%)
  surprise: 2 samples (7.7%)
  sadness: 1 samples (3.8%)
  fear: 1 samples (3.8%)
Preloading 26 val images to RAM...


Loading val: 100%|██████████| 26/26 [00:09<00:00,  2.76it/s]


VAL RAM caching completed: 26 images, ~0.02GB
Training batches: 13 (samples: 201)
Validation batches: 2 (samples: 26)

Initializing CASME II PoolFormer enhanced model...
PoolFormer feature dimension: 768
PoolFormer CASME II: 768 -> 512 -> 128 -> 7
Using Optimized Focal Loss with gamma=2.0
Per-class alpha weights: [0.053, 0.067, 0.094, 0.102, 0.106, 0.201, 0.376]
Alpha sum: 0.999
Scheduler: ReduceLROnPlateau monitoring val_f1_macro
Optimizer: AdamW (LR=1e-05)
Scheduler: ReduceLROnPlateau (patience=3)
Criterion: Optimized Focal Loss

Starting CASME II PoolFormer enhanced training...
Training configuration: 50 epochs

Epoch 1/50


...

Train - Loss: 0.0313, F1: 0.8153, Acc: 0.8259
Val   - Loss: 0.1526, F1: 0.2435, Acc: 0.3846
Time  - Epoch: 10.9s, LR: 1.00e-06
Progress: 96.0% | Best F1: 0.2847 | ETA: 0.4min

Epoch 49/50


CASME II Validation Epoch 49/50: 100%|██████████| 2/2 [00:00<00:00,  2.48it/s, Val Loss=0.0386]


Train - Loss: 0.0322, F1: 0.8480, Acc: 0.8507
Val   - Loss: 0.1529, F1: 0.2435, Acc: 0.3846
Time  - Epoch: 10.9s, LR: 1.00e-06
Progress: 98.0% | Best F1: 0.2847 | ETA: 0.2min

Epoch 50/50


CASME II Validation Epoch 50/50: 100%|██████████| 2/2 [00:00<00:00,  2.49it/s, Val Loss=0.0389]


Train - Loss: 0.0298, F1: 0.8757, Acc: 0.8806
Val   - Loss: 0.1526, F1: 0.2232, Acc: 0.3462
Time  - Epoch: 10.9s, LR: 1.00e-06
Progress: 100.0% | Best F1: 0.2847 | ETA: 0.0min

CASME II POOLFORMER BASELINE TRAINING COMPLETED
Training time: 9.3 minutes
Epochs completed: 50
Best validation F1: 0.2847 (epoch 5)
Final train F1: 0.8757
Final validation F1: 0.2232

Exporting enhanced training documentation...
Enhanced training documentation saved successfully: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/results/02_05_poolformer_casme2-af/training_logs/casme2_poolformer_training_history.json
Experiment details: Optimized Focal Loss loss
  Gamma: 2.0, Alpha Sum: 0.999
Model variant: sail/poolformer_m48 (73M)

Next: Cell 3 - CASME II PoolFormer Enhanced Evaluation
Enhanced training pipeline completed successfully!


In [ ]:
# @title Cell 3: CASME II PoolFormer Baseline Evaluation with Comprehensive Analysis

# File: 02_05_PoolFormer_CASME2_AF_Cell3.py
# Location: experiments/02_05_PoolFormer_CASME2-AF.ipynb
# Purpose: Comprehensive evaluation framework for trained CASME II PoolFormer micro-expression recognition model

import os
import time
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# Evaluation specific imports
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    classification_report, confusion_matrix,
    roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
import pickle
import warnings
warnings.filterwarnings('ignore')

# Enhanced test dataset for CASME II evaluation
class CASME2DatasetEvaluation(Dataset):
    """Enhanced CASME II test dataset with comprehensive evaluation support"""

    def __init__(self, split_metadata, dataset_root, transform=None, split='test', use_ram_cache=True):
        self.metadata = split_metadata[split]['samples']
        self.dataset_root = dataset_root
        self.transform = transform
        self.split = split
        self.use_ram_cache = use_ram_cache
        self.images = []
        self.labels = []
        self.sample_ids = []
        self.emotions = []
        self.subjects = []
        self.cached_images = []

        print(f"Loading CASME II {split} dataset for evaluation...")

        # Process metadata for evaluation
        for sample in self.metadata:
            image_path = os.path.join(dataset_root, split, sample['image_filename'])
            self.images.append(image_path)
            self.labels.append(CLASS_TO_IDX[sample['emotion']])
            self.sample_ids.append(sample['sample_id'])
            self.emotions.append(sample['emotion'])
            self.subjects.append(sample['subject'])

        print(f"Loaded {len(self.images)} CASME II {split} samples for evaluation")
        self._print_evaluation_distribution()

        # RAM caching for fast evaluation
        if self.use_ram_cache:
            self._preload_to_ram_evaluation()

    def _print_evaluation_distribution(self):
        """Print comprehensive distribution for evaluation analysis"""
        if len(self.labels) == 0:
            print("No test samples found!")
            return

        label_counts = {}
        subject_counts = {}

        for label, subject in zip(self.labels, self.subjects):
            label_counts[label] = label_counts.get(label, 0) + 1
            subject_counts[subject] = subject_counts.get(subject, 0) + 1

        print("Test set class distribution:")
        for label, count in sorted(label_counts.items()):
            class_name = CASME2_CLASSES[label]
            percentage = (count / len(self.labels)) * 100
            print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

        print(f"Test set covers {len(subject_counts)} subjects")

        # Check for missing classes
        missing_classes = []
        for i, class_name in enumerate(CASME2_CLASSES):
            if i not in label_counts:
                missing_classes.append(class_name)

        if missing_classes:
            print(f"Missing classes in test set: {missing_classes}")

    def _preload_to_ram_evaluation(self):
        """RAM preloading optimized for evaluation"""
        print(f"Preloading {len(self.images)} test images to RAM...")

        max_workers = min(mp.cpu_count(), 6)
        valid_images = 0
        self.cached_images = [None] * len(self.images)

        # Simple loading for evaluation stability
        for i, img_path in enumerate(tqdm(self.images, desc="Loading test images")):
            try:
                image = Image.open(img_path).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
                self.cached_images[i] = image
                valid_images += 1
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                # Create neutral placeholder
                self.cached_images[i] = Image.new('RGB', (224, 224), (128, 128, 128))

        ram_usage_gb = len(self.cached_images) * 224 * 224 * 3 * 4 / 1e9
        print(f"Test RAM caching completed: {valid_images} valid images, ~{ram_usage_gb:.2f}GB")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if self.use_ram_cache and self.cached_images[idx] is not None:
            image = self.cached_images[idx].copy()
        else:
            try:
                image = Image.open(self.images[idx]).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
            except:
                image = Image.new('RGB', (224, 224), (128, 128, 128))

        if self.transform:
            image = self.transform(image)

        return (image, self.labels[idx], self.sample_ids[idx],
                self.emotions[idx], self.subjects[idx], os.path.basename(self.images[idx]))

# CASME II evaluation configuration
EVALUATION_CONFIG_CASME2 = {
    'model_type': 'PoolFormer_CASME2_Baseline',
    'task_type': 'micro_expression_recognition',
    'num_classes': 7,
    'class_names': CASME2_CLASSES,
    'checkpoint_file': 'casme2_poolformer_best_f1.pth',
    'dataset_name': 'CASME_II',
    'input_size': '224x224',
    'evaluation_protocol': 'stratified_split'
}

print("CASME II PoolFormer Baseline Evaluation Framework")
print("=" * 60)
print(f"Model: {EVALUATION_CONFIG_CASME2['model_type']}")
print(f"Task: {EVALUATION_CONFIG_CASME2['task_type']}")
print(f"Classes: {EVALUATION_CONFIG_CASME2['class_names']}")
print(f"Input size: {EVALUATION_CONFIG_CASME2['input_size']}")

def extract_logits_safe_casme2(outputs_all):
    """Robust logits extraction for CASME II PoolFormer model"""
    if isinstance(outputs_all, torch.Tensor):
        return outputs_all
    if isinstance(outputs_all, (tuple, list)):
        for item in outputs_all:
            if isinstance(item, torch.Tensor):
                return item
    if isinstance(outputs_all, dict):
        for key in ('logits', 'logit', 'predictions', 'outputs', 'scores'):
            value = outputs_all.get(key)
            if isinstance(value, torch.Tensor):
                return value
        # Fallback to first tensor value
        for value in outputs_all.values():
            if isinstance(value, torch.Tensor):
                return value
    raise RuntimeError("Unable to extract tensor logits from CASME II PoolFormer model output")

def load_trained_model_casme2(checkpoint_path, device):
    """Load trained CASME II PoolFormer model with comprehensive compatibility"""
    print(f"Loading trained CASME II PoolFormer model from: {checkpoint_path}")

    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

    # Multiple loading approaches for maximum compatibility
    checkpoint = None
    loading_method = "unknown"

    try:
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        loading_method = "standard"
    except Exception as e1:
        try:
            checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
            loading_method = "weights_only_false"
        except Exception as e2:
            try:
                import pickle
                with open(checkpoint_path, 'rb') as f:
                    checkpoint = pickle.load(f)
                loading_method = "pickle"
            except Exception as e3:
                raise RuntimeError(f"All loading methods failed: {e1}, {e2}, {e3}")

    print(f"Checkpoint loaded using: {loading_method}")

    # Initialize CASME II PoolFormer model
    model = PoolFormerCASME2Baseline(
        num_classes=EVALUATION_CONFIG_CASME2['num_classes'],
        dropout_rate=CASME2_POOLFORMER_CONFIG['dropout_rate']
    ).to(device)

    # Load state dict with fallback approaches
    state_dict = checkpoint.get('model_state_dict', checkpoint)

    try:
        model.load_state_dict(state_dict, strict=True)
        print("Model state loaded with strict=True")
    except Exception as e:
        print(f"Strict loading failed, trying non-strict: {str(e)[:100]}...")
        try:
            missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)
            if missing_keys or unexpected_keys:
                print(f"Non-strict loading: Missing {len(missing_keys)}, Unexpected {len(unexpected_keys)}")
            else:
                print("Model state loaded with strict=False (no key mismatches)")
        except Exception as e2:
            raise RuntimeError(f"Both loading approaches failed: {e2}")

    model.eval()

    # Extract training information
    training_info = {
        'best_val_f1': float(checkpoint.get('best_f1', 0.0)),
        'best_val_loss': float(checkpoint.get('best_loss', float('inf'))),
        'best_val_accuracy': float(checkpoint.get('best_acc', 0.0)),
        'best_epoch': int(checkpoint.get('epoch', 0)) + 1,
        'model_checkpoint': EVALUATION_CONFIG_CASME2['checkpoint_file'],
        'num_classes': EVALUATION_CONFIG_CASME2['num_classes'],
        'config': checkpoint.get('casme2_config', {})
    }

    print(f"Model loaded successfully:")
    print(f"  Best validation F1: {training_info['best_val_f1']:.4f}")
    print(f"  Best validation accuracy: {training_info['best_val_accuracy']:.4f}")
    print(f"  Best epoch: {training_info['best_epoch']}")
    print(f"  Model classes: {EVALUATION_CONFIG_CASME2['num_classes']}")

    return model, training_info

def run_model_inference_casme2(model, test_loader, device):
    """Run CASME II PoolFormer model inference with comprehensive tracking"""
    print("Running CASME II PoolFormer model inference on test set...")

    model.eval()
    all_predictions = []
    all_probabilities = []
    all_labels = []
    all_sample_ids = []
    all_emotions = []
    all_subjects = []
    all_filenames = []

    inference_start = time.time()

    with torch.no_grad():
        for batch_idx, (images, labels, sample_ids, emotions, subjects, filenames) in enumerate(
            tqdm(test_loader, desc="CASME II Inference")):

            images = images.to(device)

            # Forward pass with robust output extraction
            try:
                outputs_raw = model(images)
                outputs = extract_logits_safe_casme2(outputs_raw)
            except Exception as e:
                print(f"Error in model forward pass: {e}")
                outputs = model(images)
                if not isinstance(outputs, torch.Tensor):
                    outputs = outputs[0] if isinstance(outputs, (list, tuple)) else outputs

            # Validate output shape for 7 CASME II classes
            if outputs.shape[1] != 7:
                print(f"Warning: Expected 7 classes output, got {outputs.shape[1]}")

            # Get probabilities and predictions
            probabilities = torch.softmax(outputs, dim=1)
            predictions = torch.argmax(probabilities, dim=1)

            # Store results (CPU for memory efficiency)
            all_predictions.extend(predictions.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_sample_ids.extend(sample_ids)
            all_emotions.extend(emotions)
            all_subjects.extend(subjects)
            all_filenames.extend(filenames)

    inference_time = time.time() - inference_start

    # Convert to arrays
    predictions_array = np.array(all_predictions)
    probabilities_array = np.array(all_probabilities)
    labels_array = np.array(all_labels)

    print(f"CASME II inference completed: {len(predictions_array)} samples in {inference_time:.2f}s")

    # Analyze prediction distribution
    unique_predictions, pred_counts = np.unique(predictions_array, return_counts=True)
    print(f"Predicted classes: {[CASME2_CLASSES[i] for i in unique_predictions]}")

    unique_labels, label_counts = np.unique(labels_array, return_counts=True)
    print(f"True classes in test: {[CASME2_CLASSES[i] for i in unique_labels]}")

    return {
        'predictions': predictions_array,
        'probabilities': probabilities_array,
        'labels': labels_array,
        'sample_ids': all_sample_ids,
        'emotions': all_emotions,
        'subjects': all_subjects,
        'filenames': all_filenames,
        'inference_time': inference_time,
        'samples_count': len(predictions_array)
    }

def analyze_wrong_predictions_casme2(inference_results):
    """Comprehensive wrong predictions analysis for CASME II"""
    print("Analyzing wrong predictions for CASME II micro-expression recognition...")

    predictions = inference_results['predictions']
    labels = inference_results['labels']
    sample_ids = inference_results['sample_ids']
    emotions = inference_results['emotions']
    subjects = inference_results['subjects']
    filenames = inference_results['filenames']

    # Find wrong predictions
    wrong_mask = predictions != labels
    wrong_indices = np.where(wrong_mask)[0]

    # Organize by true emotion class
    wrong_predictions_by_class = {}
    subject_error_analysis = {}

    for class_name in CASME2_CLASSES:
        wrong_predictions_by_class[class_name] = []

    # Analyze wrong predictions
    for idx in wrong_indices:
        true_label = labels[idx]
        pred_label = predictions[idx]
        sample_id = sample_ids[idx]
        emotion = emotions[idx]
        subject = subjects[idx]
        filename = filenames[idx]

        true_class = CASME2_CLASSES[true_label]
        pred_class = CASME2_CLASSES[pred_label]

        wrong_info = {
            'sample_id': sample_id,
            'filename': filename,
            'subject': subject,
            'true_label': int(true_label),
            'true_class': true_class,
            'predicted_label': int(pred_label),
            'predicted_class': pred_class,
            'emotion': emotion
        }

        wrong_predictions_by_class[true_class].append(wrong_info)

        # Subject error tracking
        if subject not in subject_error_analysis:
            subject_error_analysis[subject] = {'total': 0, 'wrong': 0, 'errors': []}
        subject_error_analysis[subject]['wrong'] += 1
        subject_error_analysis[subject]['errors'].append(wrong_info)

    # Count total samples per subject
    for subject in subjects:
        if subject in subject_error_analysis:
            subject_error_analysis[subject]['total'] += 1
        else:
            subject_error_analysis[subject] = {'total': 1, 'wrong': 0, 'errors': []}

    # Calculate error rates per subject
    for subject in subject_error_analysis:
        total = subject_error_analysis[subject]['total']
        wrong = subject_error_analysis[subject]['wrong']
        subject_error_analysis[subject]['error_rate'] = wrong / total if total > 0 else 0.0

    # Summary statistics
    total_wrong = len(wrong_indices)
    total_samples = len(predictions)
    error_rate = (total_wrong / total_samples) * 100

    # Confusion patterns analysis
    confusion_patterns = {}
    for idx in wrong_indices:
        true_label = labels[idx]
        pred_label = predictions[idx]
        pattern = f"{CASME2_CLASSES[true_label]}_to_{CASME2_CLASSES[pred_label]}"
        confusion_patterns[pattern] = confusion_patterns.get(pattern, 0) + 1

    analysis_results = {
        'analysis_metadata': {
            'evaluation_timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
            'model_type': EVALUATION_CONFIG_CASME2['model_type'],
            'dataset': EVALUATION_CONFIG_CASME2['dataset_name'],
            'total_samples': int(total_samples),
            'total_wrong_predictions': int(total_wrong),
            'overall_error_rate': float(error_rate)
        },
        'wrong_predictions_by_class': wrong_predictions_by_class,
        'subject_error_analysis': subject_error_analysis,
        'confusion_patterns': confusion_patterns,
        'error_summary': {
            class_name: len(wrong_predictions_by_class[class_name])
            for class_name in CASME2_CLASSES
        }
    }

    return analysis_results

def calculate_comprehensive_metrics_casme2(inference_results):
    """Calculate comprehensive evaluation metrics for CASME II micro-expression recognition"""
    print("Calculating comprehensive metrics for CASME II micro-expression recognition...")

    predictions = inference_results['predictions']
    probabilities = inference_results['probabilities']
    labels = inference_results['labels']

    if len(predictions) == 0:
        raise ValueError("No predictions to evaluate!")

    # Identify available classes in test set
    unique_test_labels = sorted(np.unique(labels))
    unique_predictions = sorted(np.unique(predictions))

    print(f"Test set contains labels: {[CASME2_CLASSES[i] for i in unique_test_labels]}")
    print(f"Model predicted classes: {[CASME2_CLASSES[i] for i in unique_predictions]}")

    # Basic metrics
    accuracy = accuracy_score(labels, predictions)

    # Macro metrics (only for available classes)
    precision, recall, f1, support = precision_recall_fscore_support(
        labels, predictions, labels=unique_test_labels, average='macro', zero_division=0
    )

    print(f"Macro F1 (available classes): {f1:.4f}")

    # Per-class metrics (all 7 classes)
    precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
        labels, predictions, labels=range(7), average=None, zero_division=0
    )

    # Confusion matrix
    cm = confusion_matrix(labels, predictions, labels=range(7))

    # Multi-class AUC (only for classes with test samples)
    auc_scores = {}
    fpr_dict = {}
    tpr_dict = {}

    try:
        labels_binarized = label_binarize(labels, classes=range(7))

        for i, class_name in enumerate(CASME2_CLASSES):
            if i in unique_test_labels and len(np.unique(labels_binarized[:, i])) > 1:
                fpr, tpr, _ = roc_curve(labels_binarized[:, i], probabilities[:, i])
                auc_score = auc(fpr, tpr)
                auc_scores[class_name] = float(auc_score)
                fpr_dict[class_name] = fpr.tolist()
                tpr_dict[class_name] = tpr.tolist()
            else:
                auc_scores[class_name] = 0.0
                fpr_dict[class_name] = [0.0, 1.0]
                tpr_dict[class_name] = [0.0, 0.0]

        # Macro AUC for available classes
        available_auc_scores = [auc_scores[CASME2_CLASSES[i]] for i in unique_test_labels]
        macro_auc = float(np.mean(available_auc_scores)) if available_auc_scores else 0.0

    except Exception as e:
        print(f"Warning: AUC calculation failed: {e}")
        auc_scores = {class_name: 0.0 for class_name in CASME2_CLASSES}
        macro_auc = 0.0

    # Subject-level analysis
    subjects = inference_results['subjects']
    subject_performance = {}

    for subject in set(subjects):
        subject_mask = [s == subject for s in subjects]
        subject_predictions = predictions[subject_mask]
        subject_labels = labels[subject_mask]

        if len(subject_predictions) > 0:
            subject_acc = accuracy_score(subject_labels, subject_predictions)
            subject_performance[subject] = {
                'accuracy': float(subject_acc),
                'samples': int(len(subject_predictions)),
                'correct': int(np.sum(subject_predictions == subject_labels))
            }

    # Comprehensive results
    comprehensive_results = {
        'evaluation_metadata': {
            'model_type': EVALUATION_CONFIG_CASME2['model_type'],
            'dataset': EVALUATION_CONFIG_CASME2['dataset_name'],
            'evaluation_timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
            'num_classes': EVALUATION_CONFIG_CASME2['num_classes'],
            'class_names': EVALUATION_CONFIG_CASME2['class_names'],
            'test_samples': int(len(labels)),
            'available_classes': [CASME2_CLASSES[i] for i in unique_test_labels],
            'missing_classes': [CASME2_CLASSES[i] for i in range(7) if i not in unique_test_labels]
        },

        'overall_performance': {
            'accuracy': float(accuracy),
            'macro_precision': float(precision),
            'macro_recall': float(recall),
            'macro_f1': float(f1),
            'macro_auc': macro_auc
        },

        'per_class_performance': {},

        'confusion_matrix': cm.tolist(),

        'subject_level_performance': subject_performance,

        'roc_analysis': {
            'auc_scores': auc_scores,
            'fpr_curves': fpr_dict,
            'tpr_curves': tpr_dict
        },

        'inference_performance': {
            'total_time_seconds': float(inference_results['inference_time']),
            'average_time_ms_per_sample': float(inference_results['inference_time'] * 1000 / len(labels))
        }
    }

    # Per-class performance details
    for i, class_name in enumerate(CASME2_CLASSES):
        comprehensive_results['per_class_performance'][class_name] = {
            'precision': float(precision_per_class[i]),
            'recall': float(recall_per_class[i]),
            'f1_score': float(f1_per_class[i]),
            'support': int(support_per_class[i]),
            'auc': auc_scores[class_name],
            'in_test_set': i in unique_test_labels
        }

    return comprehensive_results

def save_evaluation_results_casme2(evaluation_results, wrong_predictions_results, results_dir):
    """Save comprehensive evaluation results for CASME II"""
    os.makedirs(results_dir, exist_ok=True)

    # Save main evaluation results
    results_file = f"{results_dir}/casme2_poolformer_evaluation_results.json"
    with open(results_file, 'w') as f:
        json.dump(evaluation_results, f, indent=2, default=str)

    # Save wrong predictions analysis
    wrong_predictions_file = f"{results_dir}/casme2_poolformer_wrong_predictions.json"
    with open(wrong_predictions_file, 'w') as f:
        json.dump(wrong_predictions_results, f, indent=2, default=str)

    print(f"Evaluation results saved:")
    print(f"  Main results: {results_file}")
    print(f"  Wrong predictions: {wrong_predictions_file}")

    return results_file, wrong_predictions_file

# Main evaluation execution
try:
    print("Starting CASME II PoolFormer Baseline comprehensive evaluation...")

    # Create test dataset
    print("Creating CASME II test dataset...")
    casme2_test_dataset = CASME2DatasetEvaluation(
        split_metadata=GLOBAL_CONFIG_CASME2['metadata'],
        dataset_root=GLOBAL_CONFIG_CASME2['test_path'].replace('/test', ''),
        transform=GLOBAL_CONFIG_CASME2['transform_val'],
        split='test',
        use_ram_cache=True
    )

    if len(casme2_test_dataset) == 0:
        raise ValueError("No test samples found! Check test data path.")

    casme2_test_loader = DataLoader(
        casme2_test_dataset,
        batch_size=CASME2_POOLFORMER_CONFIG['batch_size'],
        shuffle=False,
        num_workers=CASME2_POOLFORMER_CONFIG['num_workers'],
        pin_memory=True
    )

    # Load trained model
    checkpoint_path = f"{GLOBAL_CONFIG_CASME2['checkpoint_root']}/{EVALUATION_CONFIG_CASME2['checkpoint_file']}"
    casme2_model, training_info = load_trained_model_casme2(checkpoint_path, GLOBAL_CONFIG_CASME2['device'])

    # Run inference
    inference_results = run_model_inference_casme2(casme2_model, casme2_test_loader, GLOBAL_CONFIG_CASME2['device'])

    # Calculate comprehensive metrics
    evaluation_results = calculate_comprehensive_metrics_casme2(inference_results)

    # Analyze wrong predictions
    wrong_predictions_results = analyze_wrong_predictions_casme2(inference_results)

    # Add training information
    evaluation_results['training_information'] = training_info

    # Save results
    results_dir = f"{GLOBAL_CONFIG_CASME2['results_root']}/evaluation_results"
    results_file, wrong_file = save_evaluation_results_casme2(
        evaluation_results, wrong_predictions_results, results_dir
    )

    # Display comprehensive results
    print("\n" + "=" * 60)
    print("CASME II POOLFORMER BASELINE EVALUATION RESULTS")
    print("=" * 60)

    # Overall performance
    overall = evaluation_results['overall_performance']
    print(f"Overall Performance (Macro - Available Classes):")
    print(f"  Accuracy:  {overall['accuracy']:.4f}")
    print(f"  Precision: {overall['macro_precision']:.4f}")
    print(f"  Recall:    {overall['macro_recall']:.4f}")
    print(f"  F1 Score:  {overall['macro_f1']:.4f}")
    print(f"  AUC:       {overall['macro_auc']:.4f}")

    # Per-class performance
    print(f"\nPer-Class Performance:")
    for class_name, metrics in evaluation_results['per_class_performance'].items():
        in_test = "Present" if metrics['in_test_set'] else "Missing"
        print(f"  {class_name} [{in_test}]: F1={metrics['f1_score']:.4f}, "
              f"AUC={metrics['auc']:.4f}, Support={metrics['support']}")

    # Training vs test comparison
    print(f"\nTraining vs Test Performance:")
    training_f1 = training_info['best_val_f1']
    training_acc = training_info['best_val_accuracy']
    test_f1 = overall['macro_f1']
    test_acc = overall['accuracy']

    print(f"  Training Val F1:  {training_f1:.4f}")
    print(f"  Test F1:          {test_f1:.4f}")
    print(f"  F1 Difference:    {training_f1 - test_f1:+.4f}")
    print(f"  Training Val Acc: {training_acc:.4f}")
    print(f"  Test Accuracy:    {test_acc:.4f}")
    print(f"  Acc Difference:   {training_acc - test_acc:+.4f}")
    print(f"  Best Epoch:       {training_info['best_epoch']}")

    # Wrong predictions summary
    print(f"\n" + "=" * 40)
    print("WRONG PREDICTIONS ANALYSIS")
    print("=" * 40)

    wrong_meta = wrong_predictions_results['analysis_metadata']
    print(f"Total wrong predictions: {wrong_meta['total_wrong_predictions']} / {wrong_meta['total_samples']}")
    print(f"Overall error rate: {wrong_meta['overall_error_rate']:.2f}%")

    print(f"\nErrors by True Class:")
    for class_name, error_count in wrong_predictions_results['error_summary'].items():
        if error_count > 0:
            wrong_samples = wrong_predictions_results['wrong_predictions_by_class'][class_name]
            print(f"  {class_name}: {error_count} errors")
            for sample in wrong_samples[:3]:  # Show first 3
                print(f"    - {sample['filename']} -> predicted as {sample['predicted_class']}")
            if len(wrong_samples) > 3:
                print(f"    ... and {len(wrong_samples) - 3} more")

    # Subject-level analysis
    print(f"\nSubject-Level Performance:")
    subject_perfs = list(evaluation_results['subject_level_performance'].items())
    subject_perfs.sort(key=lambda x: x[1]['accuracy'], reverse=True)
    for subject, perf in subject_perfs[:5]:  # Show top 5
        print(f"  {subject}: {perf['accuracy']:.3f} ({perf['correct']}/{perf['samples']})")

    # Most common confusion patterns
    print(f"\nMost Common Confusion Patterns:")
    patterns = sorted(wrong_predictions_results['confusion_patterns'].items(),
                     key=lambda x: x[1], reverse=True)
    for pattern, count in patterns[:3]:
        print(f"  {pattern}: {count} cases")

    print(f"\nInference Performance:")
    print(f"  Total time: {inference_results['inference_time']:.2f}s")
    print(f"  Speed: {evaluation_results['inference_performance']['average_time_ms_per_sample']:.1f} ms/sample")

    print(f"\nMissing Classes: {evaluation_results['evaluation_metadata']['missing_classes']}")

    print("\n" + "=" * 60)
    print("CASME II POOLFORMER BASELINE EVALUATION COMPLETED")
    print("=" * 60)

except Exception as e:
    print(f"Evaluation failed: {e}")
    import traceback
    traceback.print_exc()

finally:
    # Memory cleanup
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

print("Next: Analysis and comparison with literature baselines")

CASME II PoolFormer Baseline Evaluation Framework
Model: PoolFormer_CASME2_Baseline
Task: micro_expression_recognition
Classes: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
Input size: 224x224
Starting CASME II PoolFormer Baseline comprehensive evaluation...
Creating CASME II test dataset...
Loading CASME II test dataset for evaluation...
Loaded 28 CASME II test samples for evaluation
Test set class distribution:
  others: 10 samples (35.7%)
  disgust: 7 samples (25.0%)
  happiness: 4 samples (14.3%)
  repression: 3 samples (10.7%)
  surprise: 3 samples (10.7%)
  sadness: 1 samples (3.6%)
Test set covers 16 subjects
Missing classes in test set: ['fear']
Preloading 28 test images to RAM...


Loading test images: 100%|██████████| 28/28 [00:10<00:00,  2.60it/s]


Test RAM caching completed: 28 valid images, ~0.02GB
Loading trained CASME II PoolFormer model from: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/models/02_05_poolformer_casme2-af/casme2_poolformer_best_f1.pth
Checkpoint loaded using: standard
PoolFormer feature dimension: 768
PoolFormer CASME II: 768 -> 512 -> 128 -> 7
Model state loaded with strict=True
Model loaded successfully:
  Best validation F1: 0.2847
  Best validation accuracy: 0.4231
  Best epoch: 5
  Model classes: 7
Running CASME II PoolFormer model inference on test set...


CASME II Inference: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]

CASME II inference completed: 28 samples in 1.93s
Predicted classes: ['others', 'disgust', 'happiness', 'surprise']
True classes in test: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness']
Calculating comprehensive metrics for CASME II micro-expression recognition...
Test set contains labels: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness']
Model predicted classes: ['others', 'disgust', 'happiness', 'surprise']
Macro F1 (available classes): 0.2126
Analyzing wrong predictions for CASME II micro-expression recognition...
Evaluation results saved:
  Main results: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/results/02_05_poolformer_casme2-af/evaluation_results/casme2_poolformer_evaluation_results.json
  Wrong predictions: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/results/02_05_poolformer_casme2-af/evaluation_results/casme2_poolformer_wrong_predictions.json

CASME II POOLFORMER BA

In [ ]:
# @title Cell 4: CASME II PoolFormer Baseline Confusion Matrix Generation

# File: 02_05_PoolFormer_CASME2_AF_Cell4.py
# Location: experiments/02_05_PoolFormer_CASME2-AF.ipynb
# Purpose: Generate professional confusion matrix and comprehensive analysis for CASME II micro-expression recognition
# Dependencies: Trained model evaluation results from Cell 3

import json
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from datetime import datetime

print("CASME II PoolFormer Baseline Confusion Matrix Generation")
print("=" * 60)

# Project paths configuration
PROJECT_ROOT = "/content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project"
RESULTS_ROOT = f"{PROJECT_ROOT}/results/02_05_poolformer_casme2-af"

def find_evaluation_json_files_casme2(results_path):
    """Find CASME II evaluation JSON files"""
    json_files = {}

    eval_dir = f"{results_path}/evaluation_results"

    if os.path.exists(eval_dir):
        # Look for main evaluation results
        eval_files = glob.glob(f"{eval_dir}/casme2_poolformer_evaluation_results.json")
        if eval_files:
            json_files['main'] = eval_files[0]
            print(f"Found CASME II evaluation file: {os.path.basename(eval_files[0])}")

        # Look for wrong predictions analysis
        wrong_files = glob.glob(f"{eval_dir}/casme2_poolformer_wrong_predictions.json")
        if wrong_files:
            json_files['wrong_predictions'] = wrong_files[0]
            print(f"Found wrong predictions file: {os.path.basename(wrong_files[0])}")

        if not json_files:
            print(f"WARNING: No evaluation results found in {eval_dir}")
            print("Make sure Cell 3 (evaluation) has been executed first!")
    else:
        print(f"ERROR: Evaluation directory not found: {eval_dir}")

    return json_files

def load_evaluation_results_casme2(json_path):
    """Load and parse CASME II evaluation results JSON"""
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
        print(f"Successfully loaded evaluation results from: {os.path.basename(json_path)}")
        return data
    except Exception as e:
        print(f"ERROR loading {json_path}: {str(e)}")
        return None

def calculate_weighted_f1_casme2(per_class_performance):
    """Calculate weighted F1 score for CASME II micro-expression classes"""
    # Only count classes that have test samples (support > 0)
    total_support = sum([class_data['support'] for class_data in per_class_performance.values()
                        if class_data['support'] > 0])

    if total_support == 0:
        return 0.0

    weighted_f1 = 0.0

    for class_name, class_data in per_class_performance.items():
        if class_data['support'] > 0:  # Only include classes with test samples
            weight = class_data['support'] / total_support
            weighted_f1 += class_data['f1_score'] * weight

    return weighted_f1

def calculate_balanced_accuracy_casme2(confusion_matrix):
    """
    Calculate balanced accuracy for CASME II 7-class micro-expression recognition
    Handles classes with zero support (missing in test set)
    """
    cm = np.array(confusion_matrix)
    n_classes = cm.shape[0]

    per_class_balanced_acc = []

    # Find classes with actual test samples
    classes_with_samples = []
    for i in range(n_classes):
        if cm[i, :].sum() > 0:  # Class has test samples
            classes_with_samples.append(i)

    for i in classes_with_samples:
        # True positives, false negatives, false positives, true negatives for class i
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp  # Sum of row i minus diagonal
        fp = cm[:, i].sum() - tp  # Sum of column i minus diagonal
        tn = cm.sum() - tp - fn - fp  # Total minus TP, FN, FP

        # Calculate sensitivity and specificity for class i
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

        # Per-class balanced accuracy
        class_balanced_acc = (sensitivity + specificity) / 2
        per_class_balanced_acc.append(class_balanced_acc)

    # Overall balanced accuracy (mean of available classes)
    balanced_acc = np.mean(per_class_balanced_acc) if per_class_balanced_acc else 0.0

    return balanced_acc

def determine_text_color_casme2(color_value, threshold=0.5):
    """Determine optimal text color based on background intensity"""
    return 'white' if color_value > threshold else 'black'

def analyze_missing_classes_casme2(data):
    """Analyze missing classes in CASME II test set"""
    meta = data['evaluation_metadata']
    available_classes = meta.get('available_classes', [])
    missing_classes = meta.get('missing_classes', [])

    print(f"Class Analysis:")
    print(f"  Available in test: {available_classes}")
    print(f"  Missing from test: {missing_classes}")

    return {
        'available': available_classes,
        'missing': missing_classes,
        'total_classes': len(meta['class_names'])
    }

def create_confusion_matrix_plot_casme2(data, output_path):
    """Create professional confusion matrix visualization for CASME II micro-expression recognition"""

    # Extract data
    meta = data['evaluation_metadata']
    class_names = meta['class_names']
    cm = np.array(data['confusion_matrix'], dtype=int)
    overall = data['overall_performance']
    per_class = data['per_class_performance']

    print(f"Processing confusion matrix for CASME II classes: {class_names}")
    print(f"Confusion matrix shape: {cm.shape}")

    # Calculate comprehensive metrics
    macro_f1 = overall.get('macro_f1', 0.0)
    accuracy = overall.get('accuracy', 0.0)
    weighted_f1 = calculate_weighted_f1_casme2(per_class)
    balanced_acc = calculate_balanced_accuracy_casme2(cm)

    print(f"Calculated metrics - Macro F1: {macro_f1:.4f}, Weighted F1: {weighted_f1:.4f}, "
          f"Balanced Acc: {balanced_acc:.4f}, Accuracy: {accuracy:.4f}")

    # Row-wise normalization for percentage display
    row_sums = cm.sum(axis=1, keepdims=True)
    with np.errstate(divide='ignore', invalid='ignore'):
        cm_pct = np.divide(cm, row_sums, where=(row_sums!=0))
        cm_pct = np.nan_to_num(cm_pct)

    # Create visualization with appropriate size for 7 classes
    fig, ax = plt.subplots(figsize=(12, 10))

    # Color scheme optimized for micro-expression research
    cmap = 'Blues'

    # Create heatmap with improved color scaling
    im = ax.imshow(cm_pct, interpolation='nearest', cmap=cmap, vmin=0.0, vmax=0.8)

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('True Class Percentage', rotation=270, labelpad=15, fontsize=11)

    # Annotate cells with count and percentage
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            count = cm[i, j]

            # Handle percentage calculation for classes with 0 samples
            if row_sums[i, 0] > 0:
                percentage = cm_pct[i, j] * 100
                text = f"{count}\n{percentage:.1f}%"
            else:
                text = f"{count}\n(N/A)"  # Class has no test samples

            # Determine text color based on cell intensity
            cell_value = cm_pct[i, j]
            text_color = determine_text_color_casme2(cell_value, threshold=0.4)

            ax.text(j, i, text, ha="center", va="center",
                   color=text_color, fontsize=9, fontweight='bold')

    # Configure axes with micro-expression class names
    ax.set_xticks(np.arange(len(class_names)))
    ax.set_yticks(np.arange(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=10)
    ax.set_yticklabels(class_names, fontsize=10)
    ax.set_xlabel("Predicted Label", fontsize=12, fontweight='bold')
    ax.set_ylabel("True Label", fontsize=12, fontweight='bold')

    # Add note about missing classes if any
    missing_classes = meta.get('missing_classes', [])
    if missing_classes:
        note_text = f"Note: Missing classes in test set: {', '.join(missing_classes)}"
        ax.text(0.02, 0.98, note_text, transform=ax.transAxes, fontsize=9,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

    # Create comprehensive title for micro-expression research
    title = f"CASME II Micro-Expression Recognition - PoolFormer\n"
    title += f"Acc: {accuracy:.4f}  |  Macro F1: {macro_f1:.4f}  |  Weighted F1: {weighted_f1:.4f}  |  Balanced Acc: {balanced_acc:.4f}"
    ax.set_title(title, fontsize=12, pad=25, fontweight='bold')

    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)

    print(f"Confusion matrix saved to: {os.path.basename(output_path)}")

    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1,
        'balanced_accuracy': balanced_acc,
        'missing_classes': missing_classes
    }

def create_per_class_performance_chart_casme2(data, output_path):
    """Create per-class performance visualization for CASME II"""
    per_class = data['per_class_performance']
    class_names = data['evaluation_metadata']['class_names']

    # Extract metrics for each class
    classes = []
    f1_scores = []
    precisions = []
    recalls = []
    supports = []
    in_test_flags = []

    for class_name in class_names:
        class_data = per_class[class_name]
        classes.append(class_name)
        f1_scores.append(class_data['f1_score'])
        precisions.append(class_data['precision'])
        recalls.append(class_data['recall'])
        supports.append(class_data['support'])
        in_test_flags.append(class_data['in_test_set'])

    # Create grouped bar chart
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

    x = np.arange(len(classes))
    width = 0.25

    # Top plot: F1, Precision, Recall
    bars1 = ax1.bar(x - width, f1_scores, width, label='F1 Score', alpha=0.8, color='steelblue')
    bars2 = ax1.bar(x, precisions, width, label='Precision', alpha=0.8, color='orange')
    bars3 = ax1.bar(x + width, recalls, width, label='Recall', alpha=0.8, color='green')

    ax1.set_xlabel('Emotion Classes', fontweight='bold')
    ax1.set_ylabel('Score', fontweight='bold')
    ax1.set_title('CASME II Per-Class Performance Metrics - PoolFormer', fontweight='bold', pad=20)
    ax1.set_xticks(x)
    ax1.set_xticklabels(classes, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    ax1.set_ylim(0, 1.0)

    # Add value labels on bars
    for bars in [bars1, bars2, bars3]:
        for bar, in_test in zip(bars, in_test_flags):
            height = bar.get_height()
            if in_test:
                ax1.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                           xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)
            else:
                # Mark missing classes
                ax1.annotate('N/A', xy=(bar.get_x() + bar.get_width() / 2, height),
                           xytext=(0, 3), textcoords="offset points", ha='center', va='bottom',
                           fontsize=8, color='red', fontweight='bold')

    # Bottom plot: Support (sample count)
    bars4 = ax2.bar(x, supports, color='purple', alpha=0.7)
    ax2.set_xlabel('Emotion Classes', fontweight='bold')
    ax2.set_ylabel('Number of Test Samples', fontweight='bold')
    ax2.set_title('CASME II Test Set Class Distribution', fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(classes, rotation=45, ha='right')
    ax2.grid(axis='y', alpha=0.3)

    # Add value labels on support bars
    for bar, support in zip(bars4, supports):
        height = bar.get_height()
        ax2.annotate(f'{int(support)}', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)

    print(f"Per-class performance chart saved to: {os.path.basename(output_path)}")

def generate_performance_summary_casme2(evaluation_data, wrong_predictions_data=None):
    """Generate comprehensive performance summary for CASME II"""

    print("\n" + "=" * 60)
    print("CASME II MICRO-EXPRESSION RECOGNITION PERFORMANCE SUMMARY")
    print("=" * 60)

    # Overall performance
    overall = evaluation_data['overall_performance']
    meta = evaluation_data['evaluation_metadata']

    print(f"Dataset: {meta['dataset']}")
    print(f"Test samples: {meta['test_samples']}")
    print(f"Model: {meta['model_type']}")
    print(f"Evaluation date: {meta['evaluation_timestamp']}")

    print(f"\nOverall Performance:")
    print(f"  Accuracy:         {overall['accuracy']:.4f}")
    print(f"  Macro Precision:  {overall['macro_precision']:.4f}")
    print(f"  Macro Recall:     {overall['macro_recall']:.4f}")
    print(f"  Macro F1:         {overall['macro_f1']:.4f}")
    print(f"  Macro AUC:        {overall['macro_auc']:.4f}")

    # Per-class performance
    print(f"\nPer-Class Performance:")
    per_class = evaluation_data['per_class_performance']

    print(f"{'Class':<12} {'F1':<8} {'Precision':<10} {'Recall':<8} {'AUC':<8} {'Support':<8} {'In Test'}")
    print("-" * 65)

    for class_name, metrics in per_class.items():
        in_test = "Yes" if metrics['in_test_set'] else "No"
        print(f"{class_name:<12} {metrics['f1_score']:<8.4f} {metrics['precision']:<10.4f} "
              f"{metrics['recall']:<8.4f} {metrics['auc']:<8.4f} {metrics['support']:<8} {in_test}")

    # Training vs test performance
    if 'training_information' in evaluation_data:
        training = evaluation_data['training_information']
        print(f"\nTraining vs Test Comparison:")
        print(f"  Training Val F1:  {training['best_val_f1']:.4f}")
        print(f"  Test F1:          {overall['macro_f1']:.4f}")
        print(f"  Performance Gap:  {training['best_val_f1'] - overall['macro_f1']:+.4f}")
        print(f"  Best Epoch:       {training['best_epoch']}")

    # Class imbalance analysis
    missing_classes = meta.get('missing_classes', [])
    available_classes = meta.get('available_classes', [])

    print(f"\nClass Availability Analysis:")
    print(f"  Available classes: {len(available_classes)}/7")
    print(f"  Missing classes: {missing_classes if missing_classes else 'None'}")

    # Wrong predictions summary if available
    if wrong_predictions_data:
        wrong_meta = wrong_predictions_data['analysis_metadata']
        print(f"\nError Analysis:")
        print(f"  Total errors: {wrong_meta['total_wrong_predictions']}/{wrong_meta['total_samples']}")
        print(f"  Error rate: {wrong_meta['overall_error_rate']:.2f}%")

        # Top confusion patterns
        patterns = wrong_predictions_data.get('confusion_patterns', {})
        if patterns:
            print(f"\nTop Confusion Patterns:")
            sorted_patterns = sorted(patterns.items(), key=lambda x: x[1], reverse=True)[:3]
            for pattern, count in sorted_patterns:
                print(f"  {pattern}: {count} cases")

    print(f"\nInference Performance:")
    inference = evaluation_data['inference_performance']
    print(f"  Total time: {inference['total_time_seconds']:.2f}s")
    print(f"  Speed: {inference['average_time_ms_per_sample']:.1f} ms/sample")

# Find evaluation JSON files
json_files = find_evaluation_json_files_casme2(RESULTS_ROOT)

if not json_files:
    print(f"ERROR: No evaluation JSON files found in {RESULTS_ROOT}")
    print("Make sure Cell 3 (evaluation) has been executed first!")
else:
    print(f"Found {len(json_files)} evaluation file(s)")

# Create output directory
output_dir = f"{RESULTS_ROOT}/confusion_matrix_analysis"
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Process evaluation results
results_summary = {}
generated_files = []

if 'main' in json_files:
    # Load main evaluation data
    eval_data = load_evaluation_results_casme2(json_files['main'])

    # Load wrong predictions data if available
    wrong_data = None
    if 'wrong_predictions' in json_files:
        wrong_data = load_evaluation_results_casme2(json_files['wrong_predictions'])

    if eval_data is not None:
        try:
            # Analyze missing classes
            class_analysis = analyze_missing_classes_casme2(eval_data)

            # Generate confusion matrix
            cm_output_path = os.path.join(output_dir, "confusion_matrix_CASME2_PoolFormer.png")
            metrics = create_confusion_matrix_plot_casme2(eval_data, cm_output_path)
            generated_files.append(cm_output_path)

            # Generate per-class performance chart
            perf_output_path = os.path.join(output_dir, "per_class_performance_CASME2_PoolFormer.png")
            create_per_class_performance_chart_casme2(eval_data, perf_output_path)
            generated_files.append(perf_output_path)

            results_summary['casme2'] = metrics
            results_summary['casme2']['class_analysis'] = class_analysis

            print(f"SUCCESS: Visualization files generated successfully")

        except Exception as e:
            print(f"ERROR: Failed to generate visualizations: {str(e)}")
            import traceback
            traceback.print_exc()

        # Generate comprehensive summary
        generate_performance_summary_casme2(eval_data, wrong_data)

    else:
        print("ERROR: Could not load evaluation data")
else:
    print("ERROR: No main evaluation results found")

# Final summary
if generated_files:
    print(f"\n" + "=" * 60)
    print("CASME II POOLFORMER CONFUSION MATRIX GENERATION COMPLETED")
    print("=" * 60)

    print(f"Generated visualization files:")
    for file_path in generated_files:
        filename = os.path.basename(file_path)
        print(f"  {filename}")

    if 'casme2' in results_summary:
        casme2 = results_summary['casme2']
        print(f"\nFinal Performance Summary:")
        print(f"  Accuracy:       {casme2['accuracy']:.4f}")
        print(f"  Macro F1:       {casme2['macro_f1']:.4f}")
        print(f"  Weighted F1:    {casme2['weighted_f1']:.4f}")
        print(f"  Balanced Acc:   {casme2['balanced_accuracy']:.4f}")

        if 'class_analysis' in casme2:
            analysis = casme2['class_analysis']
            print(f"  Available classes: {len(analysis['available'])}/{analysis['total_classes']}")
            print(f"  Missing classes: {len(analysis['missing'])}")

    print(f"\nFiles saved in: {output_dir}")
    print(f"Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

else:
    print(f"\nERROR: No visualizations were generated")
    print("Please check:")
    print("1. Cell 3 evaluation results exist")
    print("2. JSON file structure is correct")
    print("3. No file permission issues")

print("\nCell 4 completed - CASME II PoolFormer confusion matrix analysis generated")

CASME II PoolFormer Baseline Confusion Matrix Generation
Found CASME II evaluation file: casme2_poolformer_evaluation_results.json
Found wrong predictions file: casme2_poolformer_wrong_predictions.json
Found 2 evaluation file(s)
Successfully loaded evaluation results from: casme2_poolformer_evaluation_results.json
Successfully loaded evaluation results from: casme2_poolformer_wrong_predictions.json
Class Analysis:
  Available in test: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness']
  Missing from test: ['fear']
Processing confusion matrix for CASME II classes: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
Confusion matrix shape: (7, 7)
Calculated metrics - Macro F1: 0.2126, Weighted F1: 0.3247, Balanced Acc: 0.5360, Accuracy: 0.3571
Confusion matrix saved to: confusion_matrix_CASME2_PoolFormer.png
Per-class performance chart saved to: per_class_performance_CASME2_PoolFormer.png
SUCCESS: Visualization files generated successfully